In [1]:
from kafka import KafkaConsumer
import nltk
import re
nltk.download('punkt')
import string
import pyspark
from pyspark import SparkContext
import numpy as np
from operator import add

[nltk_data] Downloading package punkt to /home/akki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
consumer = KafkaConsumer(bootstrap_servers='localhost:9092',auto_offset_reset="latest",group_id="test4",consumer_timeout_ms=10000)
consumer.subscribe("rahul")
sc = SparkContext()

In [3]:
from pyspark.mllib.classification import SVMModel
svm=SVMModel.load(sc,path = "BDA/ML_MODEL_svm3/ml_model_svm3")

In [4]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
dt=DecisionTreeModel.load(sc,path = "BDA/ML_MODEL_dt4/ml_model_dt4")

In [5]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
gb=GradientBoostedTreesModel.load(sc,path = "BDA/ML_MODEL_gb3/ml_model_gb3")

In [6]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
lr=LogisticRegressionModel.load(sc,path = "BDA/ML_MODEL_lr3/ml_model_lr3")

In [7]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
rf=RandomForestModel.load(sc,path = "BDA/ML_MODEL_rf3/ml_model_rf3")

In [8]:
from gensim.models import FastText
model = FastText.load("BDA/FT_MODEL_3/ft_model_3")


In [9]:
%config IPCompleter.greedy=True


def buildWordVector(tokens):
    size=10
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        vec += model.wv[word].reshape((1, size))
        count += 1.
    if count != 0:
        vec /= count
    return vec

In [10]:



def tokenize_(tweet):

    tweet_low2 = str(tweet).lower()
    tweet_low3 = tweet_low2.replace(r"\n","")
   
    punctuations = '''!()-[]{};:'"\,<>./?$%^&*_~'''
#     print(tweet_low2)
    tweet = ""
   
    tweet_low = tweet_low3
    if(tweet_low[0:4]=="b'rt" or tweet_low[0:4]=="b\"rt"):
        filtered_tweet = tweet_low[4:]
    else:
        filtered_tweet = tweet_low[2:]
    for e in filtered_tweet:
        if e not in punctuations:
            tweet+=e
    
    tweet_list = tweet.split(" ")
    tweet = ""
    
    for e in tweet_list:
        if(e!=""):
            if e[0]!='@' and e[0]!='#' and e[0:5]!="https" and e[0:4]!="http":
                if(tweet!=""):
                    tweet += e + " "
                else:
                    tweet = e + " "
    tokens = nltk.word_tokenize(tweet)
    return tokens




for message in consumer:
    tokens = tokenize_(message.value)
    print("Sentence:")
    print(message.value)
    print("Tokens:")
    print(tokens)
    if(tokens==[]):
        continue
    sentence_vector = buildWordVector(tokens) 
    predictions = [0,0,0,0,0]
    
    predictions[0]=svm.predict(sc.parallelize(sentence_vector)).collect()[0]
    predictions[1]=gb.predict(sc.parallelize(sentence_vector)).collect()[0]
    predictions[2]=rf.predict(sc.parallelize(sentence_vector)).collect()[0]
    predictions[3]=dt.predict(sc.parallelize(sentence_vector)).collect()[0]
    predictions[4]=lr.predict(sc.parallelize(sentence_vector)).collect()[0]
    
    ones = 0
    t = sc.parallelize(predictions).map(lambda x: (x,1)).reduceByKey(add)
    print("Label Counts")
    print(t.collect())
    r = sc.parallelize(predictions).filter(lambda x: x==0).map(lambda x: (x,1)).reduceByKey(add)
    if(r.collect()!=[]):
        if(int(r.collect()[0][1])>2):
            print("Final Label : Negative Sentiment")
        else:
            print("Final Label : Positive Sentiment")
    else:
        print("Final Label : Positive Sentiment")
    print("\n")
    print("\n\n")

Sentence:
b'     Rahul Gandhi  ,          https://t.co/20FVFjTmMS'
Tokens:
['rahul', 'gandhi']
Label Counts
[(1, 5)]
Final Label : Positive Sentiment





Sentence:
b'RT @sardesairajdeep: Probably the most in depth interview that @RahulGandhi has done.. combative and tough talking.. https://t.co/MNlZM06mJ'
Tokens:
['probably', 'the', 'most', 'in', 'depth', 'interview', 'that', 'has', 'done', 'combative', 'and', 'tough', 'talking']
Label Counts
[(0.0, 2), (1, 3)]
Final Label : Positive Sentiment





Sentence:
b'RT @IndiaToday: #RahulGandhiToIndiaToday\nCongress president @RahulGandhi spoke on his sense of the popular mood and his blueprint for the e'
Tokens:
['president', 'spoke', 'on', 'his', 'sense', 'of', 'the', 'popular', 'mood', 'and', 'his', 'blueprint', 'for', 'the', 'e']
Label Counts
[(0.0, 1), (1, 4)]
Final Label : Positive Sentiment





Sentence:
b'     Rahul Gandhi  ,          https://t.co/cWVOpcBRt4'
Tokens:
['rahul', 'gandhi']
Label Counts
[(1, 5)]
Final Label : Positive 

KeyboardInterrupt: 